In [2]:
import torch
from torchvision import models, transforms
from PIL import Image
import os

# 1. 定义模型架构
model = models.vgg16(pretrained=False)  # 使用 VGG16 架构
model.classifier[6] = torch.nn.Linear(4096, 2)  # 输出2个类别：炎症治疗有效（1）和无炎症治疗效果（0）

# 2. 将模型移到 GPU（如果有可用的 GPU）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 如果有 GPU，使用 GPU；否则使用 CPU
model = model.to(device)

# 3. 加载保存的模型权重到设备
model.load_state_dict(torch.load(r"E:\code\Jupyter\final_repo\best_model.pth", map_location=device))  # 加载模型权重到 GPU 或 CPU
model.eval()  # 切换为评估模式

# 4. 定义图像预处理操作
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图像大小
    transforms.ToTensor(),  # 转换为 Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
])

# 5. 测试文件夹中的所有图片
folder_path = r'E:\code\Jupyter\final_repo\pics\Anti-inflammatory'  # 替换为你的图片文件夹路径
correct = 0
total = 0

# 遍历文件夹中的所有图片
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # 确保是图片文件
        img_path = os.path.join(folder_path, filename)
        
        # 加载并确保图片是 RGB 格式
        img = Image.open(img_path).convert('RGB')

        # 对图像进行预处理并增加 batch 维度
        img_tensor = transform(img).unsqueeze(0).to(device)  # 将图片移到 GPU

        # 6. 将图片输入模型进行推理
        with torch.no_grad():  # 禁用梯度计算
            outputs = model(img_tensor)
            _, predicted = torch.max(outputs, 1)  # 获取预测的类别

        # 7. 真实标签是 1，比较预测结果
        true_label = 1  # 所有图片的标签都是 1
        total += 1
        if predicted.item() == true_label:
            correct += 1

# 8. 输出准确率
accuracy = (correct / total) * 100 if total > 0 else 0
print(f'Accuracy: {accuracy:.2f}%')


e:\code\Jupyter\Dive-into-DL-PyTorch\ENV_DIR\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\code\Jupyter\Dive-into-DL-PyTorch\ENV_DIR\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_17860\1111899027.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the defau

Accuracy: 0.00%
